### Multilevel parking system
- Number of levels/floor will be: **N**
- On each level **x** number parking slots will be there

#### APIs
```
park(reg_no, color)
unpark(reg_no)
fetch_slots_by_reg_no(reg_no)
fetch_slots_by_color(red)
```

- Filling of slots can be sequential or random
- Design should be pluggable

In [16]:
from enum import IntEnum

In [101]:
class Status(IntEnum):
    UNAVIALABLE = -1
    UNOCCUPIED = 0
    OCCUPIED = 1

class Slot:
    def __init__(self, floor, no):
        self.floor = floor
        self.no = no
        self.comb = (floor, no)
        self.id = int("{0}{1:0=3d}".format(floor, no))
        self.status = Status.UNOCCUPIED
        self.reg_no = None
        self.color = None
        
    def __repr__(self):
        return "{0} with Status: {1} & Reg no: {2}".format(self.id, self.status, self.reg_no)
    
    def park(self, reg_no, color):
        self.status = Status.OCCUPIED
        self.reg_no = reg_no
        self.color = color
    
    def unpark(self):
        self.status = Status.UNOCCUPIED
        self.reg_no = None
        self.color = None

In [102]:
s1 = Slot(1, 4)
s1

1004 with Status: 0 & Reg no: None

In [103]:
s1.park('43r233', 'red')
s1

1004 with Status: 1 & Reg no: 43r233

In [104]:
s1.unpark()
s1

1004 with Status: 0 & Reg no: None

In [151]:
class ParkingLot:
    def __init__(self, floors, slots):
        self.floor_count = floors
        self.slot_count = slots
        self.capacity = floors * slots
        self.occupied = 0
        self.occupied_slots = []
        self.vacancy = []
        self.data = {
            i: {
                "status": Status.UNOCCUPIED,
                "slots": [Slot(i, j) for j in range(1, slots+1)]
            }
            for i in range(1, floors+1)
        }
        self._prepare_vacancy()

    def __repr__(self):
        return "Parking Lot with {0} floors and {1} slots per floor.".format(self.floor_count, self.slot_count)
    
    def _prepare_vacancy(self):
        data = []
        for f in self.data:
            data.extend([i.comb for i in self.data[f]['slots']])
        self.vacancy = data
        
    def fetch_slot_by_reg_no(self, reg_no):
        x = list(filter(lambda x: x[1].reg_no == reg_no, enumerate(self.occupied_slots)))
        if x:
            return x[0]
        else:
            raise Exception("Vehicle not found")
            
    def fetch_slots_by_color(self, color):
        return list(filter(lambda x: x.color == color, self.occupied_slots))
    
    def park(self, reg_no, color):
        if self.occupied < self.capacity:
            slot = self.vacancy.pop(0)
            obj = self.data[slot[0]]['slots'][slot[1]-1]
            obj.park(reg_no, color)
            self.occupied_slots.append(obj)
            self.occupied += 1
        else:
            raise Exception("Parking Full")
            
    def unpark(self, reg_no):
        if self.occupied:
            n, slot = self.fetch_slot_by_reg_no(reg_no)
            self.vacancy.append(slot.comb)
            self.occupied_slots.pop(n)
            self.occupied -= 1
            slot.unpark()
        else:
            raise Exception("Parking Lot is Empty")
            
            

In [152]:
p = ParkingLot(4, 10)
p.capacity = 3

In [153]:
p.park('322e12e', 'red')
p.park('322e1', 'green')
p.park('322ee', 'blue')
p.park('322e134', 'yellow')

Exception: Parking Full

In [154]:
p.data

{1: {'status': <Status.UNOCCUPIED: 0>,
  'slots': [1001 with Status: 1 & Reg no: 322e12e,
   1002 with Status: 1 & Reg no: 322e1,
   1003 with Status: 1 & Reg no: 322ee,
   1004 with Status: 0 & Reg no: None,
   1005 with Status: 0 & Reg no: None,
   1006 with Status: 0 & Reg no: None,
   1007 with Status: 0 & Reg no: None,
   1008 with Status: 0 & Reg no: None,
   1009 with Status: 0 & Reg no: None,
   1010 with Status: 0 & Reg no: None]},
 2: {'status': <Status.UNOCCUPIED: 0>,
  'slots': [2001 with Status: 0 & Reg no: None,
   2002 with Status: 0 & Reg no: None,
   2003 with Status: 0 & Reg no: None,
   2004 with Status: 0 & Reg no: None,
   2005 with Status: 0 & Reg no: None,
   2006 with Status: 0 & Reg no: None,
   2007 with Status: 0 & Reg no: None,
   2008 with Status: 0 & Reg no: None,
   2009 with Status: 0 & Reg no: None,
   2010 with Status: 0 & Reg no: None]},
 3: {'status': <Status.UNOCCUPIED: 0>,
  'slots': [3001 with Status: 0 & Reg no: None,
   3002 with Status: 0 & Reg

In [155]:
p.vacancy

[(1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10)]

In [156]:
p.unpark('322e1')

In [157]:
p.fetch_slot_by_reg_no("322ee")

(1, 1003 with Status: 1 & Reg no: 322ee)

In [158]:
p.fetch_slots_by_color("red")

[1001 with Status: 1 & Reg no: 322e12e]